In [1]:
#import libraries
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

# for Mac errors
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.ticker as ticker
import numpy as np

In [2]:
# load data
full_cbsa = pd.read_csv("cbsa2.csv")
full_cbsa.head(5)

,reference_period,type_of_service,aggregation_level,cbsa,cbsatitle,number_of_fee_for_service_beneficiaries,number_of_providers,average_number_of_users_per_provider,percentage_of_users_out_of_ffs_beneficiaries,number_of_users,...,average_number_of_providers_per_cbsa_dual_color,average_number_of_providers_per_cbsa_description,number_of_dual_eligible_users_dual_color,number_of_dual_eligible_users_description,percentage_of_dual_eligible_users_out_of_total_users_dual_color,percentage_of_dual_eligible_users_out_of_total_users_description,percentage_of_dual_eligible_users_out_of_dual_eligible_ffs_beneficiaries_dual_color,percentage_of_dual_eligible_users_out_of_dual_eligible_ffs_beneficiaries_dual_color_description,total_payment_dual_color,total_payment_description
0,2015-01-01 to 2015-12-31,Ambulance (Emergency & Non-Emergency),NATION + TERRITORIES,--ALL--,--ALL--,"37,359,009","9,078",424.34,10.31%,"3,852,199",...,,,,,,,,,,
1,2015-01-01 to 2015-12-31,Ambulance (Emergency & Non-Emergency),CBSA,10100,"Aberdeen, SD","7,526",3,231,9.21%,693,...,BLUE 1,Lowest 25% (Less than 5.00 Providers),BLUE 2,Second Lowest 25% (166 - 349 Dual Eligible Users),BLUE 2,Second Lowest 25% (21.67% - 26.95% of Total Us...,BLUE 3,Third Lowest 25% (17.06% - 19.32% of Total FFS...,BLUE 2,"Second Lowest 25% ($525,929.51 - $1,042,852.56 )"
2,2015-01-01 to 2015-12-31,Ambulance (Emergency & Non-Emergency),CBSA,10140,"Aberdeen, WA","17,349",18,110.44,11.46%,"1,988",...,BLUE 4,Top 25% Excl. Extreme Values (16.00 - 32.49 Pr...,BLUE 3,Third Lowest 25% (350 - 845 Dual Eligible Users),BLUE 4,Top 25% Excl. Extreme Values (33.58% - 51.44% ...,BLUE 4,Top 25% Excl. Extreme Values (19.33% - 27.78% ...,BLUE 3,"Third Lowest 25% ($1,042,852.57 - $2,686,138.28 )"
3,2015-01-01 to 2015-12-31,Ambulance (Emergency & Non-Emergency),CBSA,10180,"Abilene, TX","25,198",11,271.09,11.83%,"2,982",...,BLUE 3,Third Lowest 25% (9.00 - 15.99 Providers),BLUE 4,"Top 25% Excl. Extreme Values (846 - 1,864 Dual...",BLUE 3,Third Lowest 25% (26.96% - 33.57% of Total Users),BLUE 4,Top 25% Excl. Extreme Values (19.33% - 27.78% ...,BLUE 3,"Third Lowest 25% ($1,042,852.57 - $2,686,138.28 )"
4,2015-01-01 to 2015-12-31,Ambulance (Emergency & Non-Emergency),CBSA,10220,"Ada, OK","7,570",5,104.8,6.92%,524,...,BLUE 2,Second Lowest 25% (5.00 - 8.99 Providers),BLUE 2,Second Lowest 25% (166 - 349 Dual Eligible Users),BLUE 4,Top 25% Excl. Extreme Values (33.58% - 51.44% ...,BLUE 1,Lowest 25% (Less than 13.69% of Total FFS Bene...,BLUE 1,"Lowest 25% (Less than $525,929.51 )"


In [3]:
full_cbsa.shape

(163035, 35)

## Data Preparation

Clean and prepare data for data analysis


    - change misleading field values
    - remap categorical as numerical
    - standardize numeric variables
    - identify outliers

In [4]:
cbsa = full_cbsa.iloc[:, :15]
cbsa.head()

,reference_period,type_of_service,aggregation_level,cbsa,cbsatitle,number_of_fee_for_service_beneficiaries,number_of_providers,average_number_of_users_per_provider,percentage_of_users_out_of_ffs_beneficiaries,number_of_users,average_number_of_providers_per_cbsa,number_of_dual_eligible_users,percentage_of_dual_eligible_users_out_of_total_users,percentage_of_dual_eligible_users_out_of_dual_eligible_ffs_beneficiaries,total_payment
0,2015-01-01 to 2015-12-31,Ambulance (Emergency & Non-Emergency),NATION + TERRITORIES,--ALL--,--ALL--,"37,359,009","9,078",424.34,10.31%,"3,852,199",116.56,"1,085,184",28.17%,17.24%,"$3,430,203,620.67"
1,2015-01-01 to 2015-12-31,Ambulance (Emergency & Non-Emergency),CBSA,10100,"Aberdeen, SD","7,526",3,231,9.21%,693,3,174,25.11%,18.73%,"$588,150.97"
2,2015-01-01 to 2015-12-31,Ambulance (Emergency & Non-Emergency),CBSA,10140,"Aberdeen, WA","17,349",18,110.44,11.46%,"1,988",18,675,33.95%,19.55%,"$1,786,700.58"
3,2015-01-01 to 2015-12-31,Ambulance (Emergency & Non-Emergency),CBSA,10180,"Abilene, TX","25,198",11,271.09,11.83%,"2,982",11,907,30.42%,19.76%,"$2,003,346.18"
4,2015-01-01 to 2015-12-31,Ambulance (Emergency & Non-Emergency),CBSA,10220,"Ada, OK","7,570",5,104.8,6.92%,524,5,183,34.92%,12.12%,"$359,960.14"


<table style="border:1px solid black;">
  <tr>
    <th width="200px" style="background-color: lightgrey; border:1px solid black;">Quantitative (interval)</th>
    <th width="200px" style="background-color: lightgrey; border:1px solid black;">Quantitative (ratio)   </th>
  </tr>
  <tr>
    <td style="background-color: lightblue; vertical-align: top; border:1px solid black;">reference_period</td>
    <td style="background-color: lightblue; vertical-align: top; border:1px solid black;">number_of_fee_for_service_beneficiaries,
number_of_providers,
average_number_of_users_per_provider,
percentage_of_users_out_of_ffs_beneficiaries,
number_of_users,
average_number_of_providers_per_cbsa,
number_of_dual_eligible_users,
percentage_of_dual_eligible_users_out_of_total_users,
percentage_of_dual_eligible_users_out_of_dual_eligible_ffs_beneficiaries,
total_payment
</td>
  </tr>
</table>

<table>
  <tr>
    <th width="200px" style="background-color: lightgrey; border:1px solid black;">Qualitative (nominal)</th>
    <th width="200px" style="background-color: lightgrey; border:1px solid black;">Qualitative (ordinal)</th>
  </tr>
  <tr>
    <td style="background-color: lightblue; vertical-align: top; border:1px solid black;">type_of_service,
cbsa,
cbsatitle
    <td style="background-color: lightblue; vertical-align: top; border:1px solid black;">
aggregation_level
    </td>
  </tr>
</table>

In [5]:
# remove percent and dollar signs
cbsa = cbsa.replace('%', '', regex=True)
cbsa = cbsa.replace('\$', '', regex=True)

# remove aggregate
cbsa = cbsa[cbsa['cbsa'] != '--ALL--']

In [6]:
# split reference period to start and end dates
cbsa[['start_date', 'end_date']] = cbsa['reference_period'].str.split(' to ', expand=True)
cbsa.insert(0, 'end_date', cbsa.pop('end_date'))
cbsa.insert(0, 'start_date', cbsa.pop('start_date'))
cbsa.drop(["reference_period"], axis = 1, inplace= True)

**Identifying and Converting Attributes**

In [7]:
# convert date columns to datetime
cbsa[['start_date', 'end_date']] = cbsa[['start_date', 'end_date']].apply(pd.to_datetime)

In [8]:
# convert numerical variables to integer or float
convert_numerical = [
    'number_of_fee_for_service_beneficiaries',
    'number_of_providers',
    'number_of_users',
    'number_of_dual_eligible_users',
    'average_number_of_users_per_provider',
    'percentage_of_users_out_of_ffs_beneficiaries',
    'average_number_of_providers_per_cbsa',
    'percentage_of_dual_eligible_users_out_of_total_users',
    'percentage_of_dual_eligible_users_out_of_dual_eligible_ffs_beneficiaries',
    'total_payment'
]

for column in convert_numerical:
    cbsa[column] = cbsa[column].str.replace(' ', '').str.replace(',', '')
    cbsa[column] = pd.to_numeric(cbsa[column], downcast=None)

In [9]:
# identify and convert to categorical
convert_to_category = [
    'type_of_service',
    'cbsa',
    'cbsatitle',
    'aggregation_level',
]
cbsa[convert_to_category] = cbsa[convert_to_category].astype('category')

In [10]:
# confirm datatypes
print(cbsa.dtypes)

start_date                                                                  datetime64[ns]
end_date                                                                    datetime64[ns]
type_of_service                                                                   category
aggregation_level                                                                 category
cbsa                                                                              category
cbsatitle                                                                         category
number_of_fee_for_service_beneficiaries                                              int64
number_of_providers                                                                  int64
average_number_of_users_per_provider                                               float64
percentage_of_users_out_of_ffs_beneficiaries                                       float64
number_of_users                                                                      int64

In [11]:
# statistics
cbsa.describe()

,start_date,end_date,number_of_fee_for_service_beneficiaries,number_of_providers,average_number_of_users_per_provider,percentage_of_users_out_of_ffs_beneficiaries,number_of_users,average_number_of_providers_per_cbsa,number_of_dual_eligible_users,percentage_of_dual_eligible_users_out_of_total_users,percentage_of_dual_eligible_users_out_of_dual_eligible_ffs_beneficiaries,total_payment
count,162848,162848,1.628480e+05,162848.000000,159036.000000,162848.000000,1.628480e+05,162848.000000,148123.000000,148123.000000,148123.000000,1.628480e+05
mean,2019-01-12 09:51:07.144822016,2020-01-11 15:09:18.145018624,3.876744e+04,50.072258,115.417717,12.941694,5.105435e+03,50.072258,1015.588173,23.279648,15.206498,1.158631e+07
min,2015-01-01 00:00:00,2015-12-31 00:00:00,4.070000e+02,0.000000,2.310000,0.010000,1.100000e+01,0.000000,11.000000,0.470000,0.010000,5.953100e+02
25%,2017-01-01 00:00:00,2017-12-31 00:00:00,6.180000e+03,4.000000,42.000000,2.560000,2.380000e+02,4.000000,64.000000,13.040000,3.990000,2.189874e+05
50%,2019-01-01 00:00:00,2019-12-31 00:00:00,1.196600e+04,10.000000,66.330000,5.890000,8.010000e+02,10.000000,188.000000,20.260000,9.670000,9.170519e+05
75%,2021-01-01 00:00:00,2021-12-31 00:00:00,2.689500e+04,28.000000,130.600000,15.610000,2.830000e+03,28.000000,579.000000,30.510000,20.130000,4.449768e+06
max,2023-01-01 00:00:00,2023-12-31 00:00:00,2.523066e+06,16732.000000,2581.130000,75.080000,1.421406e+06,16732.000000,231507.000000,100.000000,79.170000,6.625352e+09
std,NaN,NaN,1.176749e+05,278.108241,137.590262,15.949473,2.603557e+04,278.108241,5331.255444,13.911181,15.261371,8.285980e+07


In [12]:
# check null values
cbsa.isnull().sum()

start_date                                                                      0
end_date                                                                        0
type_of_service                                                                 0
aggregation_level                                                               0
cbsa                                                                            0
cbsatitle                                                                       0
number_of_fee_for_service_beneficiaries                                         0
number_of_providers                                                             0
average_number_of_users_per_provider                                         3812
percentage_of_users_out_of_ffs_beneficiaries                                    0
number_of_users                                                                 0
average_number_of_providers_per_cbsa                                            0
number_of_dual_e

In [13]:
# check if cbsa have sufficient users to qualify
cbsa.loc[cbsa['number_of_users'] == cbsa['number_of_users'].min(),'number_of_users']

765       11
1949      11
2815      11
3288      11
3713      11
          ..
156802    11
156975    11
159581    11
162312    11
162479    11
Name: number_of_users, Length: 320, dtype: int64

**Observation:** The lowest number of user is 11, meaning the providers should be defined by CMS. Requirement is 10 and above. There appears to be many that are close to the cutoff. There may have been technicalities that did not qualify the providers according to CMS criteria.

**Investigate `average_number_of_users_per_provider`**

In [14]:
# impute number providers of 0 with previous data
cbsa['number_of_providers'].replace(0, np.nan, inplace=True)

In [15]:
# create a subset of cbsa with complete dual data
provider_subset = ['start_date','type_of_service','cbsa','number_of_providers']

cbsa_provider = cbsa[provider_subset].sort_values('start_date').dropna().drop_duplicates(
    subset=['type_of_service', 'cbsa'], keep='last')

In [16]:
# fill missing values with the most recent available data
match_cols1 = ['cbsa','type_of_service']
cbsa = cbsa.set_index(match_cols1).fillna(cbsa_provider.set_index(match_cols1)).reset_index()

In [17]:
# cbsa.to_csv('cbsa_providernum.csv', index=False)

In [18]:
# due to provider circumstances 0
cbsa['number_of_providers'] = cbsa['number_of_providers'].fillna(0)

**Identify and remove outliers, under conditions that number of users are 10 but provider is 0**

**Investigate `number_of_dual_eligible_users`**

    - replace missing values with same data from the most recent previous reference period
    - estimate number of dual eligible using average qualifiers per type of sertice

In [19]:
# create a subset of cbsa with complete dual data
dual_subset = ['start_date','type_of_service','cbsa','number_of_dual_eligible_users']

cbsa_dual = cbsa[dual_subset].sort_values('start_date').dropna().drop_duplicates(
    subset=['type_of_service', 'cbsa'], keep='last')

In [20]:
# fill missing values with the most recent available data
match_cols2 = ['cbsa','type_of_service']
cbsa = cbsa.set_index(match_cols2).fillna(cbsa_dual.set_index(match_cols2)).reset_index()

In [21]:
# aggregate mean of percentage_of_dual_eligible_users_out_of_total_users per service type

mean_percentage_service = cbsa.groupby('type_of_service')['percentage_of_dual_eligible_users_out_of_total_users'].mean()
mean_percentage_service = mean_percentage_service.reset_index()
mean_percentage_service

,type_of_service,percentage_of_dual_eligible_users_out_of_total_users
0,Ambulance (Emergency & Non-Emergency),27.484972
1,Ambulance (Emergency),27.981974
2,Ambulance (Non-Emergency),29.584675
3,Cardiac Rehabilitation Program,12.125258
4,Chiropractic Services,8.943611
5,Clinical Laboratory (Billing Independently),18.059210
6,Dialysis,39.513165
7,Federally Qualified Health Center (FQHC),45.332903
8,Home Health,21.352746
9,Hospice,20.758439


In [22]:
match_mean_dual_service = ['type_of_service']

In [23]:
cbsa = cbsa.set_index(match_mean_dual_service).fillna(mean_percentage_service.set_index(match_mean_dual_service)).reset_index()

In [24]:
cbsa

,type_of_service,cbsa,start_date,end_date,aggregation_level,cbsatitle,number_of_fee_for_service_beneficiaries,number_of_providers,average_number_of_users_per_provider,percentage_of_users_out_of_ffs_beneficiaries,number_of_users,average_number_of_providers_per_cbsa,number_of_dual_eligible_users,percentage_of_dual_eligible_users_out_of_total_users,percentage_of_dual_eligible_users_out_of_dual_eligible_ffs_beneficiaries,total_payment
0,Ambulance (Emergency & Non-Emergency),10100,2015-01-01,2015-12-31,CBSA,"Aberdeen, SD",7526,3.0,231.00,9.21,693,3.0,174.0,25.11,18.73,588150.97
1,Ambulance (Emergency & Non-Emergency),10140,2015-01-01,2015-12-31,CBSA,"Aberdeen, WA",17349,18.0,110.44,11.46,1988,18.0,675.0,33.95,19.55,1786700.58
2,Ambulance (Emergency & Non-Emergency),10180,2015-01-01,2015-12-31,CBSA,"Abilene, TX",25198,11.0,271.09,11.83,2982,11.0,907.0,30.42,19.76,2003346.18
3,Ambulance (Emergency & Non-Emergency),10220,2015-01-01,2015-12-31,CBSA,"Ada, OK",7570,5.0,104.80,6.92,524,5.0,183.0,34.92,12.12,359960.14
4,Ambulance (Emergency & Non-Emergency),10300,2015-01-01,2015-12-31,CBSA,"Adrian, MI",16863,18.0,103.94,11.10,1871,18.0,529.0,28.27,18.54,1524406.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162843,Telemedicine,49660,2023-01-01,2023-12-31,CBSA,"Youngstown-Warren-Boardman, OH-PA",60651,13.0,74.38,1.59,967,13.0,222.0,22.96,3.89,192700.78
162844,Telemedicine,49700,2023-01-01,2023-12-31,CBSA,"Yuba City, CA",28180,39.0,45.92,6.36,1791,39.0,566.0,31.60,7.01,325781.43
162845,Telemedicine,49740,2023-01-01,2023-12-31,CBSA,"Yuma, AZ",26729,10.0,52.80,1.98,528,10.0,78.0,14.77,1.84,62069.58
162846,Telemedicine,49780,2023-01-01,2023-12-31,CBSA,"Zanesville, OH",11997,12.0,43.67,4.37,524,12.0,257.0,49.05,13.01,102256.78


In [25]:
# fill nulls of average_number_of_users_per_provider with num users/num providers
fill_avg_users_per_provider = cbsa['number_of_users']/cbsa['number_of_providers']
cbsa['average_number_of_users_per_provider'].fillna(fill_avg_users_per_provider, inplace=True)

In [26]:
# fill nulls of number_of_dual_eligible_users by average number per service type
fill_dual_users = cbsa['number_of_users']*(cbsa[
    'percentage_of_dual_eligible_users_out_of_total_users']/100)
cbsa['number_of_dual_eligible_users'].fillna(fill_dual_users, inplace=True)

In [27]:
cbsa['percentage_of_dual_eligible_users_out_of_dual_eligible_ffs_beneficiaries'] = cbsa[
    'number_of_dual_eligible_users']/cbsa['number_of_fee_for_service_beneficiaries']

In [28]:
# check null values
cbsa.isnull().sum()

type_of_service                                                             0
cbsa                                                                        0
start_date                                                                  0
end_date                                                                    0
aggregation_level                                                           0
cbsatitle                                                                   0
number_of_fee_for_service_beneficiaries                                     0
number_of_providers                                                         0
average_number_of_users_per_provider                                        0
percentage_of_users_out_of_ffs_beneficiaries                                0
number_of_users                                                             0
average_number_of_providers_per_cbsa                                        0
number_of_dual_eligible_users                                   

In [29]:
cbsa.to_csv('cbsa_providerv5.csv', index=False)